# Python Technologist Application Test

## Problem 1


[This is](https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D9%85%D8%A6%D8%A9_%D8%B1%D9%88%D8%A7%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9) a Wikipedia webpage listing the 100 best Arabic novels according to the Arab Writers Union. In case it did not open for any reason try [this](https://www.marefa.org/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D8%A7%D9%84%D9%83%D8%AA%D8%A8_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9) alternative link for the same info 

Using Python, do the following:

1. Scrap the webpage to get the books table and write it to excel file, Keeping all the content from the HTML table including Hyper-links if any.
2. Write the content to a Google sheet 


Write your code in the following cell. You are free to add cells as much as you need.

In [ ]:
! pip install requests bs4 pandas openpyxl
! pip3 install gspread pygsheets
! pip3 install --upgrade google-api-python-client oauth2client 

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D9%85%D8%A6%D8%A9_%D8%B1%D9%88%D8%A7%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9'
home='https://ar.wikipedia.org'
page = requests.get(url)

soup = BeautifulSoup(page.content)

In [3]:
table_head=[
    head.text
    for head in 
    soup.find("table", class_="wikitable").find_all('th')
]

'''

'''
def bookUrl(item,index): 
    # if  '(الصفحة غير موجودة)' not in item.td.find_next_siblings(limit = 3)[index].a['title'] :
    return home+item.td.find_next_siblings(limit = 3)[index].a['href']
    # else :
    #     return False
            

'''

''' 
def  bookName(item,index):
    return item.td.find_next_siblings(limit = 3)[index].text
    

In [4]:
table = [
    
{
table_head[0]:item.td.text,
table_head[1]:('=HYPERLINK("{}", "{}")'.format(bookUrl(item,0), bookName(item,0)) if bookUrl(item,0) else  bookName(item,0)),
table_head[2]:('=HYPERLINK("{}", "{}")'.format(bookUrl(item,1), bookName(item,1)) if bookUrl(item,1) else  bookName(item,1)),
table_head[3]:('=HYPERLINK("{}", "{}")'.format(bookUrl(item,2), bookName(item,2)) if bookUrl(item,2) else  bookName(item,2)) }
    
    
for item in 
soup.find("table", class_="wikitable").find_all('tr')
if item.td is not None
]

In [5]:
df = pd.DataFrame(table)
df.to_excel(excel_writer='Book_Table.xlsx',index=False)

In [23]:
import pygsheets

gc = pygsheets.authorize(service_file='nagwatask-db5953efd44c.json')
sh = gc.open("top_100_best_novels")
print(sh.url)
wks = sh[0]
wks.set_dataframe(df,(1,1))

https://docs.google.com/spreadsheets/d/15ERmviQNI8t-3TbESqjOlKovl2ZIrqn4oFoXTRuXpKU


## Problem 2 

Create REST APIs in Python using Flask to read (Get) and write (Post, Delete, Put) the local excel file from the previous problem. Please make sure to bundle all the API dependencies to be uasble. Deploying the API to Heroku would be a big plus.

Write your code in the following cell. You are free to add cells as much as you need.

In [7]:
! pip install flask==2.1.2  requests jsonify flask-restful pyexcel


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Nabil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from flask import Flask, request, jsonify, json
from flask_restful import Resource, Api, abort, reqparse
import pyexcel as p
from openpyxl import load_workbook

app = Flask(__name__)
api = Api(app)

### Models

In [9]:

class BooksModel:

    def __init__(self):
        self.dictionary = None
        self.fileName = "Book_Table.xlsx"
        self.sheet = p.get_sheet(file_name=self.fileName)
        self.wb = load_workbook('Book_Table.xlsx')
        self.books = self.wb.active

    def update_dict(self):
        self.wb = load_workbook('Book_Table.xlsx')
        self.books = self.wb.active
        self.dictionary = {
            self.books.cell(row, 1).value: {
                self.books.cell(1, 2).value: self.books.cell(row, 2).value.split("\"")[1:4:2]
                , self.books.cell(1, 3).value: self.books.cell(row, 3).value.split("\"")[1:4:2]
                , self.books.cell(1, 4).value: self.books.cell(row, 4).value.split("\"")[1:4:2]
            }
            for row in
            range(2, self.books.max_row + 1)
        }

    def add_book(self, book, sorting):
        book = json.loads(book)
        book_row = [
            sorting,
            '=HYPERLINK("{}", "{}")'.format(book['الرواية\n'][0], book['الرواية\n'][1]),
            '=HYPERLINK("{}", "{}")'.format(book['المؤلف\n'][0], book['المؤلف\n'][1]),
            '=HYPERLINK("{}", "{}")'.format(book['البلد\n'][0], book['البلد\n'][1])]
        self.books.append(book_row)
        self.wb.save('Book_Table.xlsx')


    def get_books(self):
        self.update_dict()
        return self.dictionary


    def get_book(self, id):
        try:
            self.update_dict()
            return self.dictionary[str(id) + "\n"]
        except:
            return False


    def delete_book(self, id):
        try:
            for i in range(1, self.books.max_row + 1):
                if self.books.cell(row=i, column=1).value == id + "\n":
                    self.books.delete_rows(i, 1)
            self.wb.save('Book_Table.xlsx')
            return True
        except:
            return False

    def edit_book(self, id, book):
        try:
            book = json.loads(book)

            for i in range(1, self.books.max_row + 1):
                if self.books.cell(row=i, column=1).value == id + "\n":
                    self.books.cell(row=i, column=2).value = '=HYPERLINK("{}", "{}")'.format(book['الرواية\n'][0],
                                                                                             book['الرواية\n'][1])
                    self.books.cell(row=i, column=3).value = '=HYPERLINK("{}", "{}")'.format(book['المؤلف\n'][0],
                                                                                             book['المؤلف\n'][1])
                    self.books.cell(row=i, column=4).value = '=HYPERLINK("{}", "{}")'.format(book['البلد\n'][0],
                                                                                             book['البلد\n'][1])
                    self.wb.save('Book_Table.xlsx')

            return book
        except:
            return False



### Controllers

In [10]:

class Book(Resource):

    def get(self, id):
        b = BooksModel()
        book = b.get_book(id)
        if book is False:
            abort(404, message="Book {} doesn't exist".format(id))

        return book

    def delete(self, id):
        b = BooksModel()
        if b.delete_book(id) is False:
            abort(404, message="Book {} doesn't exist".format(id))
        return '', 204

    def put(self, id):
        b = BooksModel()
        book = b.edit_book(id, book=request.form['book'])
        if book is False:
            abort(404, message="Book {} doesn't exist".format(id))

        return book, 201


class Books(Resource):
    def get(self):
        b = BooksModel()
        books = b.get_books()
        return jsonify(book=books)

    def post(self):
        b = BooksModel()
        book = b.add_book(request.form['book'], request.form['sorting'])

        return book, 201




In [11]:
api.add_resource(Books, '/books')
api.add_resource(Book, '/books/<id>')

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.8:5000 (Press CTRL+C to quit)


## Problem 3

Write a tool to create a PDF cover for the books, following the attached example "book-cover-sample.pdf". Considering the following:

    1. The QR code should embed the book hyperlink from Wikipedia.
    2. The QR code should be clickable to let the users acess it by clicking on it.
    3. Include all the covers in one directory and compress it in ZIP format.
    

Write your code in the following cell. You are free to add cells as much as you need.

In [12]:
! pip3 install qrcode FPDF


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Nabil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
! mkdir book_covers

A subdirectory or file book_covers already exists.


In [14]:
from enum import auto
import qrcode
from fpdf import FPDF
import arabic_reshaper
import shutil
 

def generate_image(link,path):

    path="book_covers/"+path+".png"
    img = qrcode.make(link)
    img.save(path)

    return path

def generate_cover(link,path,auther):

    pdf = FPDF()
    pdf.add_page()
    pdf.image(generate_image(link,path), x = None, y = None, type = '', link = link)
    pdf.add_font('arabic', '', 'DejaVuSansCondensed.ttf', uni=True)
    pdf.set_font('arabic', '',20 )
    pdf.cell(200, 10, txt = arabic_reshaper.reshape(path)[::-1],ln = 1, align = 'C')
    pdf.cell(w=200, h = 10, txt = arabic_reshaper.reshape(auther)[::-1], ln = 1, align = 'C')
    pdf.output("book_covers/"+path+'.pdf', 'F')


In [ ]:
for book in table:
    
    link,name=book[table_head[1]].split("\"")[1:4:2]
    name=name.split("\n")[0]
    auther=book[table_head[2]].split("\"")[3].split("\n")[0]


    generate_cover(link,name,auther)




In [24]:

shutil.make_archive("books","zip","book_covers" )

'c:\\Users\\Nabil\\OneDrive\\المستندات\\Python_Tech_-_Application_Test_1\\books.zip'